In [6]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
from random import choice
import os
from scipy.signal import savgol_filter
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\DATA\DATATABLES\time_plate_infos.pick"
time_plate_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\DATA\DATATABLES\global_hypha_infos.pick"
global_hypha_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\DATA\DATATABLES\time_hypha_infos.pick"
time_hypha_info = pd.read_pickle(path_save)
plt.style.use('presentation.mplstyle')
global_hypha_info['hypha_unique'] = global_hypha_info['hypha'].astype(str)+'_'+global_hypha_info['Plate'].astype(str)
time_hypha_info['hypha_unique'] = time_hypha_info['end'].astype(str)+'_'+time_hypha_info['Plate'].astype(str)


In [3]:
def get_branch(row):
    if row['Plate']==94:
        hyph = row['end']
        name = f'branches/ts_{hyph}_auto.npy'
        if not os.path.isfile(name):
            return(None)
        else:
            t_real = np.load(name)
            return(row['time_since_emergence'] in t_real)

In [7]:
global_hypha_info['lapse']=global_hypha_info['time_stop_growth']-global_hypha_info['time_init_growth']
time_hypha_info['distance_final_pos_smooth'] = time_hypha_info['distance_final_pos'].rolling(window = 5).mean()
time_hypha_info['new_branch'] = time_hypha_info.apply(get_branch, axis=1)

In [127]:
plates = [94,792,800,91,152]
select_time_plate = time_plate_info.loc[time_plate_info['Plate'].isin(plates)]
select_time = time_hypha_info.loc[time_hypha_info['Plate'].isin(plates)]
select_global = global_hypha_info.loc[global_hypha_info['Plate'].isin(plates)]
select_global = select_global.loc[select_global['lapse']>0]
hyphae_interest= select_global['hypha'].unique()
# hyphae_select= [hyph for hyph in hyphae_interest if len(select_time.loc[(select_time['end']==hyph)&(select_time['has_reached_final_pos']=='False')])]
select_time = select_time.loc[select_time['end'].isin(hyphae_interest)]
# select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
select_time = select_time.loc[select_time['speed'].between(50,350)]
select_time['speed'] = select_time['speed'].astype(float)


select_time = select_time.loc[select_time['distance_final_pos']>200]
select_time["int_time"] = select_time['time_since_begin_exp'].astype(int)
select_time["int_time"] = select_time["int_time"]-140*(select_time['Plate']==799)
select_time["int_time"] = select_time["int_time"]-300*(select_time['Plate']==91)
bin = 8
select_time["binned_time"] = select_time['int_time'].astype(int)//bin*bin

select_time = select_time.loc[select_time['in_ROI']=='True']

In [125]:
select_time.hist('speed',bins=30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<AxesSubplot:title={'center':'speed'}>]], dtype=object)

In [129]:
import joypy
select = select_time.loc[select_time['binned_time'].between(0,100)]
fig, axes = joypy.joyplot(select,'speed',by='binned_time')
fig.supxlabel('speed $(\mu m.s^{-1})$')
fig.supylabel('time $(h)$')
# plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.02, 0.5, 'time $(h)$')

In [135]:
len(select['hypha_unique'].unique())

2053

In [105]:
plates = [91]
select_time_plate = time_plate_info.loc[time_plate_info['Plate'].isin(plates)]
select_time = time_hypha_info.loc[time_hypha_info['Plate'].isin(plates)]
select_global = global_hypha_info.loc[global_hypha_info['Plate'].isin(plates)]
select_global = select_global.loc[select_global['lapse']>5]
select_global = select_global.loc[select_global['mean_speed']>200]

hyphae_interest= select_global['hypha_unique'].unique()
# hyphae_select= [hyph for hyph in hyphae_interest if len(select_time.loc[(select_time['end']==hyph)&(select_time['has_reached_final_pos']=='False')])]
select_time = select_time.loc[select_time['hypha_unique'].isin(hyphae_interest)]
select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
select_time = select_time.loc[select_time['speed'].between(100,350)]
select_time['speed'] = select_time['speed'].astype(float)


select_time = select_time.loc[select_time['distance_final_pos']>800]

select_time["int_time"] = select_time['time_since_emergence'].astype(float).round()
select_time["int_time2"] = select_time['time_since_begin_exp'].astype(float).round()
if plates[0]==799:
    select_time["int_time2"] = select_time["int_time2"]-140
if plates[0]==91:
    select_time["int_time2"] = select_time["int_time2"]-300
select_time = select_time.loc[select_time['in_ROI']=='True']
hyphae = select_time.groupby("hypha_unique")["timestep"].count()>=5
select_time = select_time.set_index("hypha_unique")
select_time = select_time.loc[hyphae[select_time.index]]
hyphae = select_time.index.unique()

In [98]:
ijs = [(0,0),(1,0),(2,0),(0,1),(1,1),(2,1)]

In [106]:
fig, axs = plt.subplots(3,2,figsize = (8,5))
ax = axs[2][0]
for hypha in hyphae:
    plot = select_time.loc[select_time.index==hypha]
    if len(plot)>0:
        ax.plot(plot['int_time2'],plot['speed'],alpha =0.2)
data = select_time.groupby('int_time2')['speed']

# ax.plot(data.mean(),label = 'mean')
ax.errorbar(data.mean().index,data.mean(),data.std()/np.sqrt(data.count()),label = 'mean',color='black')
lim_up= 100
ax.set_xlim((0,lim_up))
fig.supxlabel("time since begin (h)")
fig.supylabel("growth speed ($\mu m .h^{-1}$)")
ax.legend()
plt.tight_layout()
ax.set_title(f"Plate {plates[0]}",fontdict = {"fontsize" : 8})

# plt.savefig("Figure/growth_speed_radial_hyphae.svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\numpy\core\_methods.py:43: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial, where)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\numpy\core\_methods.py:39: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial, where)


Text(0.5, 1.0, 'Plate 91')

In [107]:
# dic_hyphae = {}
# dic_lim_up = {}
dic_hyphae[plates[0]]=hyphae
dic_lim_up[plates[0]]=lim_up


In [122]:
plates_list = [[94],[799],[800],[91],[152],[792]]
# select_time_plate = time_plate_info.loc[time_plate_info['Plate'].isin(plates)]
select_time = time_hypha_info
# select_global = global_hypha_info.loc[global_hypha_info['Plate'].isin(plates)]
select_global = select_global.loc[select_global['lapse']>5]
select_global = select_global.loc[select_global['mean_speed']>200]

hyphae_interest= select_global['hypha_unique'].unique()
# hyphae_select= [hyph for hyph in hyphae_interest if len(select_time.loc[(select_time['end']==hyph)&(select_time['has_reached_final_pos']=='False')])]
# select_time = select_time.loc[select_time['hypha_unique'].isin(hyphae_interest)]
select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
select_time = select_time.loc[select_time['speed'].between(100,350)]
select_time['speed'] = select_time['speed'].astype(float)
select_time = select_time.loc[select_time['distance_final_pos']>800]
select_time["int_time"] = select_time['time_since_emergence'].astype(float).round()
select_time["int_time2"] = select_time['time_since_begin_exp'].astype(float).round()
select_time["int_time2"] = select_time["int_time2"]-140*(select_time['Plate']==799)
select_time["int_time2"] = select_time["int_time2"]-300*(select_time['Plate']==91)


In [116]:
select_time['hypha_unique']

0              0_799
0              0_799
0              0_799
0              0_799
0              0_799
             ...    
86708      86708_787
93203      93203_787
97308      97308_787
107892    107892_787
107892    107892_787
Name: hypha_unique, Length: 13113, dtype: object

In [123]:
fig, axs = plt.subplots(3,2,figsize = (8,5))
for k, plates in enumerate(plates_list):
    ij = ijs[k]
    ax = axs[ij[0]][ij[1]]
    hyphae = dic_hyphae[plates[0]]
    for hypha in hyphae:
        plot = select_time.loc[select_time['hypha_unique']==hypha]
        if len(plot)>0:
            ax.plot(plot['int_time2'],plot['speed'],alpha =0.2)
    data = select_time.loc[select_time['hypha_unique'].isin(hyphae)]
    data = data.groupby('int_time2')['speed']

    # ax.plot(data.mean(),label = 'mean')
    ax.errorbar(data.mean().index,data.mean(),data.std()/np.sqrt(data.count()),label = 'mean',color='black')
    lim_up= dic_lim_up[plates[0]]
    ax.set_xlim((0,lim_up))
    fig.supxlabel("time since begin (h)")
    fig.supylabel("growth speed ($\mu m .h^{-1}$)")
    ax.legend()
    plt.tight_layout()
    ax.set_title(f"Plate {plates[0]}",fontdict = {"fontsize" : 8})
#     break

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\numpy\core\_methods.py:43: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial, where)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\numpy\core\_methods.py:39: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial, where)


In [59]:
fig, ax = plt.subplots()
for hypha in hyphae_interest:
    plot = select_time.loc[select_time.index==hypha]
    ax.plot(plot['int_time'],plot['speed'],alpha =0.1)
data = select_time.groupby('int_time')['speed']

ax.plot(data.mean())
ax.errorbar(data.mean().index,data.mean(),data.std()/np.sqrt(data.count()))
ax.set_xlim((0,60))
ax.set_xlabel("time since emergence (h)")
ax.set_ylabel("average growth speed ($\mu m .h^{-1}$)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'average growth speed ($\\mu m .h^{-1}$)')

In [20]:
fig, ax = plt.subplots()
for hypha in hyphae_interest:
    plot = select_time.loc[select_time.index==hypha]
    ax.plot(plot['int_time'],plot['speed'],alpha =0.2)
data = select_time.groupby('int_time')['speed']

ax.plot(data.mean())
ax.errorbar(data.mean().index,data.mean(),data.std()/np.sqrt(data.count()))
ax.set_xlim((0,60))
ax.set_xlabel("time since emergence (h)")
ax.set_ylabel("average growth speed ($\mu m .h^{-1}$)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'average growth speed ($\\mu m .h^{-1}$)')

In [22]:
fig, ax = plt.subplots()
for hypha in hyphae_interest:
    plot = select_time.loc[select_time.index==hypha]
    ax.plot(plot['int_time'],plot['width_root_edge'],alpha =0.2)
data = select_time.groupby('int_time')['width_root_edge']

ax.plot(data.mean())
ax.errorbar(data.mean().index,data.mean(),data.std()/np.sqrt(data.count()))
# ax.set_xlim((0,20))
ax.set_xlabel("time since emergence (h)")
ax.set_ylabel("average growth speed ($\mu m .h^{-1}$)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

DataError: No numeric types to aggregate

In [21]:
select_time['width_root_edge']

hypha_unique
950_152       NaN
950_152       NaN
950_152       NaN
1434_152     None
950_152       NaN
             ... 
28327_152     NaN
28327_152     NaN
28327_152     NaN
28327_152     NaN
28327_152     NaN
Name: width_root_edge, Length: 465, dtype: object

In [29]:
select_time['int_time'].unique()[0] in list(select_time['int_time'])

True

In [45]:
select_time['absolute_angle']

280       75.490053
288      -131.11418
280        81.16846
288     -131.550487
395      -50.748663
            ...    
9232            NaN
13240           NaN
4354            NaN
9232            NaN
13240           NaN
Name: absolute_angle, Length: 157, dtype: object

In [32]:
plot['int_time'].round()

4467    0
4467    3
4467    8
Name: int_time, dtype: int32

In [145]:
plate = 152

In [116]:
fig, ax = plt.subplots()

select_time.plot('time_since_emergence','speed',kind = 'scatter',alpha = 0.05,s = 80,ax = ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time_since_emergence', ylabel='speed'>

In [13]:
hyphae = select_time['end'].unique()
tot = 1
hyph_plot = [choice(hyphae) for i in range(tot)]

In [8]:
plate = 94

In [14]:
plt.close('all')
# hyph = hyphae[i]
i =0
hyphae = select_time['end'].unique()

hyph_plot = [choice(hyphae) for i in range(tot)]
hyph = hyph_plot[0]
# hyph = 5435
to_plot = select_time.loc[select_time['end']==hyph]
to_plot['speed_smooth'] = to_plot['speed'].rolling(window = 2).mean()
to_plot.plot('time_since_emergence','speed',xlabel = 'Time (h)',ylabel = 'speed (um.h-1)',title = f'hyph {hyph}, plate {plate}',)
# plt.vlines(t_real,np.min(to_plot['speed_smooth']),np.max(to_plot['speed_smooth']),color="red",alpha = 0.3)
i+=1

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [184]:
len(hyphae)

196